### Control

The evolution of a linear dynamical system with states $x_i$ and control inputs $u_i$ can be described by the equation:
\begin{equation}
x_{t+1} = Ax_t + Bu_t
\end{equation}
where $A$ and $B$ are the matrices controlling the (linear) state updates.

A generic control problem might try to minimize the overall cost of the inputs, subject to achieving some sort of final state, in a specified amount of time, and some maximum size of input. For example, moving the state from $x_{init}$ to 0 in $T$ time steps:

\begin{equation*}
  \begin{aligned}
    &\text{minimize} && \sum_{i = 0}^{T-1} \|u_i\|_1 \\
    &\text{subject to} && x_{i+1} = Ax_i + Bu_i &&& i = 0,..., T-1 \\
                     &  && x_0 = x_{init} \\
                     &  && x_T = 0 
  \end{aligned}
\end{equation*}

The variables are the states $x_0,..., x_T$ and the inputs $u_0,..., u_{T-1}$. $x_{init}$ is a fixed parameter.

In [1]:
import cvxpy as cp
import numpy as np
import scipy as sp

# Variable declarations

# Generate data for control problem.
np.random.seed(1)
m = 2 # number of inputs
n = 8 # number of states
T = 50 # number of time steps
alpha = 0.2
beta = 5
A = np.eye(n) + alpha*np.random.randn(n,n)
B = np.random.randn(n,m)
x_0 = beta*np.random.randn(n,1)

# Form and solve control problem.
x = cp.Variable(n, T+1)
u = cp.Variable(m, T)


# Problem construction
prob = None
opt_val = None

states = []
for t in range(T):
    cost = cp.pnorm(u[:,t], 1)
    constr = [x[:,t+1] == A*x[:,t] + B*u[:,t],
              cp.norm(u[:,t], 'inf') <= 1]
    states.append(cp.Problem(cp.Minimize(cost), constr))
# sums problem objectives and concatenates constraints.
prob = sum(states)
prob.constraints += [x[:,T] == 0, x[:,0] == x_0]


# For debugging individual problems:
if __name__ == "__main__":
    prob.solve()
    print("status:", prob.status)
    print("optimal value:", prob.value)
    print("true optimal value:", opt_val)

('status:', 'optimal')
('optimal value:', 80.98494786229705)
('true optimal value:', None)
